In [77]:
from bs4 import BeautifulSoup 
import requests 

def get_soup_by_link(link):
    response = requests.get(link) 
    return BeautifulSoup(response.text, 'html.parser')
 
def parse_rating(table_movie_rating): 
    try: 
        return table_movie_rating.tbody.tr.td.div.span.text 
    except: 
        return "Unknown" 
    
def parse_genre(table_movie_info): 
    try: 
        return table_movie_info.find('td', {'class': 'genre'}).p.a.text
    except: 
        return "Unknown" 
    
def parse_year(table_movie_info):
    try:
        return table_movie_info.find('td', {'class': 'year'}).a.text
    except:
        return "Unknown"

def parse(table_movie_info, class_name):
    try:
        return table_movie_info.find('td', {'class': class_name}).text
    except:
        return "Unknown"

def parse_min_age(td_with_data):
    try:
        return td_with_data.find('div', {'class': 'title__labels'}).contents[1].text
    except:
        return "Unknown"

def parse_director(td_with_data):
    try:
        return td_with_data.findAll('p', recursive = False)[0].text[10:]
    except:
        return "Unknown"    

def parse_actors(td_with_data):
    try:
        return td_with_data.findAll('p', recursive = False)[1].text[9:]
    except:
        return "Unknown"

main_link = 'https://afisha.tut.by/film/'
soup = get_soup_by_link(main_link)
films_li = soup.findAll('li', {'class': 'lists__li'}) 
links = []
for film_li in films_li: 
    film_links = film_li.findAll('a', {'class': 'name'}) 
    if len(film_links) != 0: 
        link = film_links[0]['href']
        if link.startswith(main_link) and '?utm_source' not in link: 
            links.append(link)

movies = {}
links_amount = len(links)
i = 1
for link in links:
    print(str(i) + " of " + str(links_amount) + ": " + link)
    i = i + 1
    
    soup = get_soup_by_link(link)
    td_with_data = soup.find('td', {'class': 'post b-event-post'}) 
    table_movie_info = td_with_data.find('table', {'class': 'movie_info'}) 
    table_movie_rating = td_with_data.find('table', {'class': 'movie_rating'}) 
    name = td_with_data.h1.string
    
    movies[name] = {}
    movies[name]['link'] = link
    movies[name]['genre'] = parse_genre(table_movie_info)
    movies[name]['year'] = parse_year(table_movie_info)
    movies[name]['country'] = parse(table_movie_info, 'author')
    movies[name]['duration'] = parse(table_movie_info, 'duration')
    movies[name]['end_date'] = parse(table_movie_info, 'date')
    movies[name]['director'] = parse_director(td_with_data) 
    movies[name]['actors'] = parse_actors(td_with_data)
    movies[name]['rating'] = parse_rating(table_movie_rating)
    movies[name]['min_age'] = parse_min_age(td_with_data)

1 of 93: https://afisha.tut.by/film/malefisenta-vladychica-tmy/
2 of 93: https://afisha.tut.by/film/tekst/
3 of 93: https://afisha.tut.by/film/oni/
4 of 93: https://afisha.tut.by/film/okey-leksi/
5 of 93: https://afisha.tut.by/film/dzhudi-2019/
6 of 93: https://afisha.tut.by/film/proshchanie-2019/
7 of 93: https://afisha.tut.by/film/maleficent_mistress_of_evil_ru_sub/
8 of 93: https://afisha.tut.by/film/zelenee-travy/
9 of 93: https://afisha.tut.by/film/predpokaz-dikaya-liga/
10 of 93: https://afisha.tut.by/film/muzey-prado-kollekciya-chudes/
11 of 93: https://afisha.tut.by/film/joker-film/
12 of 93: https://afisha.tut.by/film/gemini/
13 of 93: https://afisha.tut.by/film/everest-film/
14 of 93: https://afisha.tut.by/film/devushki-byvayut-raznye/
15 of 93: https://afisha.tut.by/film/matrica/
16 of 93: https://afisha.tut.by/film/dozhdlivyy-den-v-nyu-yorke/
17 of 93: https://afisha.tut.by/film/once-upon-a-time-in-hollywood/
18 of 93: https://afisha.tut.by/film/k-zvezdam/
19 of 93: https:/

In [78]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName("minald").getOrCreate()
sc = spark.sparkContext
pdDF = pd.DataFrame(movies).transpose()
pdDF

,actors,country,director,duration,end_date,genre,link,min_age,rating,year
Малефисента: Владычица тьмы,"Мишель Пфайффер, Анджелина Джоли, Эд Скрейн, Э...",США,Хоаким Роннинг,Unknown,В прокате до 12 ноября,Unknown,https://afisha.tut.by/film/malefisenta-vladych...,Unknown,"9,3",2019
Текст,"Александр Петров, Кристина Асмус, Иван Янковск...",Россия,Клим Шипенко,Unknown,В прокате до 27 октября,Драма,https://afisha.tut.by/film/tekst/,16+,"7,4",2019
Они,"Кэти Стивенс, Уилл Бриттейн, Лорин Алиса МакКл...",США,"Скотт Бек, Брайан Вудс",92 минуты,В прокате до 27 октября,Триллер,https://afisha.tut.by/film/oni/,18+,"5,8",2019
"Окей, Лекси!","Роуз Бирн, Александра Шипп, Адам Дивайн, Майкл...",США,"Джон Лукас, Скотт Мур",80 минут,В прокате до 24 октября,Unknown,https://afisha.tut.by/film/okey-leksi/,Unknown,Unknown,2019
Джуди,"Рене Зеллвегер, Джесси Бакли, Финн Уиттрок, Ру...",Великобритания,Руперт Гулд,118 минут,В прокате до 27 октября,Биография,https://afisha.tut.by/film/dzhudi-2019/,Unknown,Unknown,2019
Прощание,"Чжоу Шучжэнь, Аквафина, X Mayo, Лу Хун, Линь Х...",США,Лулу Ванг,100 минут,В прокате до 24 октября,Драма,https://afisha.tut.by/film/proshchanie-2019/,16+,"7,5",2019
Maleficent: Mistress of Evil (RU SUB),Unknown,Unknown,Unknown,125 минут,В прокате до 23 октября,Приключения,https://afisha.tut.by/film/maleficent_mistress...,Unknown,Unknown,2019
Зеленее травы,"Д’Арси Карден, Дот-Мари Джонс, Мэри Холлэнд, Д...",США,"Жослин ДеБоэр, Доун Люббе",95 минут,В прокате до 23 октября,Комедия,https://afisha.tut.by/film/zelenee-travy/,16+,"6,5",2019
Предпоказ. Дикая Лига,"Владимир Яглыч, Аделина Голубенко-Гизатуллина,...",Россия,"Андрей Богатырев, Арт Камачо",Unknown,В прокате до 19 октября,Боевик,https://afisha.tut.by/film/predpokaz-dikaya-liga/,Unknown,Unknown,2019
Музей Прадо: Коллекция чудес,Джереми Айронс,Италия,Валерия Паризи,Unknown,В прокате до 20 октября,Документальный,https://afisha.tut.by/film/muzey-prado-kollekc...,Unknown,Unknown,2019


In [79]:
movies_df = spark.createDataFrame(pdDF)
movies_df.show()
movies_df.createOrReplaceTempView("all_movies")
movies_df.groupBy("min_age").count().show()

+--------------------+-------------------+--------------------+---------+--------------------+--------------+--------------------+-------+-------+-------+
|              actors|            country|            director| duration|            end_date|         genre|                link|min_age| rating|   year|
+--------------------+-------------------+--------------------+---------+--------------------+--------------+--------------------+-------+-------+-------+
|Мишель Пфайффер, ...|                США|      Хоаким Роннинг|  Unknown|В прокате до 12 н...|       Unknown|https://afisha.tu...|Unknown|    9,3|   2019|
|Александр Петров,...|             Россия|        Клим Шипенко|  Unknown|В прокате до 27 о...|         Драма|https://afisha.tu...|    16+|    7,4|   2019|
|Кэти Стивенс, Уил...|                США|Скотт Бек, Брайан...|92 минуты|В прокате до 27 о...|       Триллер|https://afisha.tu...|    18+|    5,8|   2019|
|Роуз Бирн, Алекса...|                США|Джон Лукас, Скотт...| 80 мин

In [96]:
import pyspark.sql.functions as f

countries_from_rows = spark.sql("SELECT COUNTRY FROM all_movies")
countries = countries_from_rows.select(
  "COUNTRY",
  f.split("COUNTRY", ", ").alias("COUNTRY"),
  f.posexplode(f.split("COUNTRY", ", ")).alias("pos", "_COUNTRY")
  ).select("_COUNTRY")
countries.createOrReplaceTempView("countries")
country_counts = spark.sql("SELECT _COUNTRY, COUNT(*) FROM countries GROUP BY _COUNTRY ORDER BY _COUNTRY DESC")
country_counts.show()

actors_from_rows = spark.sql("SELECT actors FROM all_movies")
actors = actors_from_rows.select(
  "actors",
  f.split("actors", ", ").alias("actors"),
  f.posexplode(f.split("actors", ", ")).alias("pos", "_actors")
  ).select("_actors")
actors.createOrReplaceTempView("actors")
actors_counts = spark.sql("SELECT _actors, COUNT(*) FROM actors GROUP BY _actors ORDER BY COUNT(*) DESC")
actors_counts.show()

+---------------+--------+
|       _COUNTRY|count(1)|
+---------------+--------+
|         Япония|       2|
|        Эстония|       1|
|          Чехия|       1|
|        Франция|       4|
|            США|      24|
|         Россия|      12|
|     Португалия|       1|
|       Норвегия|       1|
|        Мексика|       1|
|    Корея Южная|       1|
|          Китай|       4|
|         Канада|       2|
|         Италия|       3|
|        Испания|       5|
|          Дания|       1|
|         Грузия|       1|
|       Германия|       4|
|Великобритания |       2|
| Великобритания|      10|
|       Бразилия|       1|
+---------------+--------+
only showing top 20 rows

+--------------------+--------+
|             _actors|count(1)|
+--------------------+--------+
|             Unknown|      38|
|      Владимир Яглыч|       3|
|         Эль Фаннинг|       3|
|          Киану Ривз|       2|
|        Ребекка Холл|       2|
|     Иван Охлобыстин|       2|
|      Лоренс Фишбёрн|       2|
|     